In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore') 
from datetime import datetime

from sklearn.preprocessing import MinMaxScaler

import statsmodels.api as sm

In [29]:
data = pd.read_csv("walmart.csv")
data

,Unnamed: 0,Store,Date,IsHoliday,Dept,Weekly_Sales,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,2010-02-05,False,1.0,24924.50,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
1,2,1,2010-02-05,False,26.0,11737.12,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
2,3,1,2010-02-05,False,17.0,13223.76,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
3,4,1,2010-02-05,False,45.0,37.44,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
4,5,1,2010-02-05,False,28.0,1085.29,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423320,423321,45,2013-06-28,False,NaN,NaN,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,NaN,NaN,B,118221
423321,423322,45,2013-07-05,False,NaN,NaN,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,NaN,NaN,B,118221
423322,423323,45,2013-07-12,False,NaN,NaN,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,NaN,NaN,B,118221
423323,423324,45,2013-07-19,False,NaN,NaN,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,NaN,NaN,B,118221


In [30]:
data['Weekly_Sales'] = data['Weekly_Sales'].apply(lambda x : x if x > 0 else np.nan)

In [31]:
data = data.drop("Unnamed: 0", axis = 1)

In [32]:
data["Date"] = pd.to_datetime(data["Date"])
data = data.set_index("Date")
data

,Store,IsHoliday,Dept,Weekly_Sales,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
Date,,,,,,,,,,,,,,,
2010-02-05,1,False,1.0,24924.50,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
2010-02-05,1,False,26.0,11737.12,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
2010-02-05,1,False,17.0,13223.76,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
2010-02-05,1,False,45.0,37.44,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
2010-02-05,1,False,28.0,1085.29,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-06-28,45,False,NaN,NaN,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,NaN,NaN,B,118221
2013-07-05,45,False,NaN,NaN,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,NaN,NaN,B,118221
2013-07-12,45,False,NaN,NaN,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,NaN,NaN,B,118221


In [33]:
data[['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']] = data[['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']].fillna(0)
data["Total_Markdown"] = data[['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']].sum(axis = 1)
data = data.drop(['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'], axis = 1)
data

,Store,IsHoliday,Dept,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,Type,Size,Total_Markdown
Date,,,,,,,,,,,
2010-02-05,1,False,1.0,24924.50,42.31,2.572,211.096358,8.106,A,151315,0.00
2010-02-05,1,False,26.0,11737.12,42.31,2.572,211.096358,8.106,A,151315,0.00
2010-02-05,1,False,17.0,13223.76,42.31,2.572,211.096358,8.106,A,151315,0.00
2010-02-05,1,False,45.0,37.44,42.31,2.572,211.096358,8.106,A,151315,0.00
2010-02-05,1,False,28.0,1085.29,42.31,2.572,211.096358,8.106,A,151315,0.00
...,...,...,...,...,...,...,...,...,...,...,...
2013-06-28,45,False,NaN,NaN,76.05,3.639,NaN,NaN,B,118221,11439.98
2013-07-05,45,False,NaN,NaN,77.50,3.614,NaN,NaN,B,118221,19254.20
2013-07-12,45,False,NaN,NaN,79.37,3.614,NaN,NaN,B,118221,8598.17


In [34]:
# Replacing values A:3 ; B:2 ; C:1
data['Type'].replace(['A', 'B', 'C'], [3,2,1], inplace=True)
# Replacing values True = 1 and False = 0
data['IsHoliday'] = data['IsHoliday'].astype(int)
data

,Store,IsHoliday,Dept,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,Type,Size,Total_Markdown
Date,,,,,,,,,,,
2010-02-05,1,0,1.0,24924.50,42.31,2.572,211.096358,8.106,3,151315,0.00
2010-02-05,1,0,26.0,11737.12,42.31,2.572,211.096358,8.106,3,151315,0.00
2010-02-05,1,0,17.0,13223.76,42.31,2.572,211.096358,8.106,3,151315,0.00
2010-02-05,1,0,45.0,37.44,42.31,2.572,211.096358,8.106,3,151315,0.00
2010-02-05,1,0,28.0,1085.29,42.31,2.572,211.096358,8.106,3,151315,0.00
...,...,...,...,...,...,...,...,...,...,...,...
2013-06-28,45,0,NaN,NaN,76.05,3.639,NaN,NaN,2,118221,11439.98
2013-07-05,45,0,NaN,NaN,77.50,3.614,NaN,NaN,2,118221,19254.20
2013-07-12,45,0,NaN,NaN,79.37,3.614,NaN,NaN,2,118221,8598.17


In [72]:
df = data[data["Dept"].notna()]
df

,Store,IsHoliday,Dept,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,Type,Size,Total_Markdown
Date,,,,,,,,,,,
2010-02-05,1,0,1.0,24924.50,42.31,2.572,211.096358,8.106,3,151315,0.00
2010-02-05,1,0,26.0,11737.12,42.31,2.572,211.096358,8.106,3,151315,0.00
2010-02-05,1,0,17.0,13223.76,42.31,2.572,211.096358,8.106,3,151315,0.00
2010-02-05,1,0,45.0,37.44,42.31,2.572,211.096358,8.106,3,151315,0.00
2010-02-05,1,0,28.0,1085.29,42.31,2.572,211.096358,8.106,3,151315,0.00
...,...,...,...,...,...,...,...,...,...,...,...
2012-10-26,45,0,13.0,26240.14,58.85,3.882,192.308899,8.667,2,118221,5247.26
2012-10-26,45,0,16.0,2660.02,58.85,3.882,192.308899,8.667,2,118221,5247.26
2012-10-26,45,0,32.0,4131.54,58.85,3.882,192.308899,8.667,2,118221,5247.26


In [73]:
df.to_csv("Walmart_1.csv")

In [74]:
df.isna().sum()

Store                0
IsHoliday            0
Dept                 0
Weekly_Sales      1358
Temperature          0
Fuel_Price           0
CPI                  0
Unemployment         0
Type                 0
Size                 0
Total_Markdown       0
dtype: int64

In [78]:
# df['Weekly_Sales'].fillna(df['Weekly_Sales'].median(), inplace=True)
df = df.interpolate(method ='linear', limit_direction ='forward')

In [79]:
df.isna().sum()

Store             0
IsHoliday         0
Dept              0
Weekly_Sales      0
Temperature       0
Fuel_Price        0
CPI               0
Unemployment      0
Type              0
Size              0
Total_Markdown    0
dtype: int64

In [11]:
# df = df.drop(["Type", "Size"], axis = 1)

In [80]:
dept_encoded = pd.get_dummies(df["Dept"])
type_encoded = pd.get_dummies(df["Type"])
type_encoded.columns = ["Type_1", "Type_2", "Type_3"]

In [86]:
df_encoded = pd.concat([df, dept_encoded , type_encoded], axis = 1)

In [87]:
df_encoded

,Store,IsHoliday,Dept,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,Type,Size,...,93.0,94.0,95.0,96.0,97.0,98.0,99.0,Type_1,Type_2,Type_3
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-05,1,0,1.0,24924.50,42.31,2.572,211.096358,8.106,3,151315,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,26.0,11737.12,42.31,2.572,211.096358,8.106,3,151315,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,17.0,13223.76,42.31,2.572,211.096358,8.106,3,151315,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,45.0,37.44,42.31,2.572,211.096358,8.106,3,151315,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,28.0,1085.29,42.31,2.572,211.096358,8.106,3,151315,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-10-26,45,0,13.0,26240.14,58.85,3.882,192.308899,8.667,2,118221,...,0,0,0,0,0,0,0,0,1,0
2012-10-26,45,0,16.0,2660.02,58.85,3.882,192.308899,8.667,2,118221,...,0,0,0,0,0,0,0,0,1,0
2012-10-26,45,0,32.0,4131.54,58.85,3.882,192.308899,8.667,2,118221,...,0,0,0,0,0,0,0,0,1,0


In [88]:
df_2 = df_encoded.drop(['Store','Dept','Type'],axis=1)

In [89]:
df_2

,IsHoliday,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,Size,Total_Markdown,1.0,2.0,...,93.0,94.0,95.0,96.0,97.0,98.0,99.0,Type_1,Type_2,Type_3
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-05,0,24924.50,42.31,2.572,211.096358,8.106,151315,0.00,1,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,0,11737.12,42.31,2.572,211.096358,8.106,151315,0.00,0,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,0,13223.76,42.31,2.572,211.096358,8.106,151315,0.00,0,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,0,37.44,42.31,2.572,211.096358,8.106,151315,0.00,0,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,0,1085.29,42.31,2.572,211.096358,8.106,151315,0.00,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-10-26,0,26240.14,58.85,3.882,192.308899,8.667,118221,5247.26,0,0,...,0,0,0,0,0,0,0,0,1,0
2012-10-26,0,2660.02,58.85,3.882,192.308899,8.667,118221,5247.26,0,0,...,0,0,0,0,0,0,0,0,1,0
2012-10-26,0,4131.54,58.85,3.882,192.308899,8.667,118221,5247.26,0,0,...,0,0,0,0,0,0,0,0,1,0


In [90]:
to_nomalize = ['Temperature','Weekly_Sales', 'Fuel_Price', 'CPI', 'Unemployment', 'Total_Markdown', "Size"]
df_2[to_nomalize] = MinMaxScaler().fit_transform(df_2[to_nomalize])

In [91]:
df_2

,IsHoliday,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,Size,Total_Markdown,1.0,2.0,...,93.0,94.0,95.0,96.0,97.0,98.0,99.0,Type_1,Type_2,Type_3
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-05,0,0.035961,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,1,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,0,0.016934,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,0,0.019079,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,0,0.000054,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,0,0.001566,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-10-26,0,0.037859,0.595988,0.706413,0.654796,0.458884,0.451136,0.032691,0,0,...,0,0,0,0,0,0,0,0,1,0
2012-10-26,0,0.003838,0.595988,0.706413,0.654796,0.458884,0.451136,0.032691,0,0,...,0,0,0,0,0,0,0,0,1,0
2012-10-26,0,0.005961,0.595988,0.706413,0.654796,0.458884,0.451136,0.032691,0,0,...,0,0,0,0,0,0,0,0,1,0


In [92]:
df_2.to_csv("walmart_2_x.csv")

In [52]:
# type_encoded = pd.get_dummies(df["Type"])
# type_encoded.columns = ["Type_1", "Type_2", "Type_3"]

In [63]:
# df_encoded = pd.concat([df, dept_encoded], axis = 1)
# df_encoded.drop(["Dept"], axis = 1, inplace = True)
# df_encoded

,Store,IsHoliday,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,Type,Size,Total_Markdown,...,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-05,1,0,24924.50,42.31,2.572,211.096358,8.106,3,151315,0.00,...,0,0,0,0,0,0,0,0,0,0
2010-02-05,1,0,11737.12,42.31,2.572,211.096358,8.106,3,151315,0.00,...,0,0,0,0,0,0,0,0,0,0
2010-02-05,1,0,13223.76,42.31,2.572,211.096358,8.106,3,151315,0.00,...,0,0,0,0,0,0,0,0,0,0
2010-02-05,1,0,37.44,42.31,2.572,211.096358,8.106,3,151315,0.00,...,0,0,0,0,0,0,0,0,0,0
2010-02-05,1,0,1085.29,42.31,2.572,211.096358,8.106,3,151315,0.00,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-10-26,45,0,26240.14,58.85,3.882,192.308899,8.667,2,118221,5247.26,...,0,0,0,0,0,0,0,0,0,0
2012-10-26,45,0,2660.02,58.85,3.882,192.308899,8.667,2,118221,5247.26,...,0,0,0,0,0,0,0,0,0,0
2012-10-26,45,0,4131.54,58.85,3.882,192.308899,8.667,2,118221,5247.26,...,0,0,0,0,0,0,0,0,0,0


In [95]:
to_nomalize = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Total_Markdown', "Size", "Weekly_Sales"]
df_encoded[to_nomalize] = MinMaxScaler().fit_transform(df_encoded[to_nomalize])
df_encoded

,Store,IsHoliday,Dept,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,Type,Size,...,93.0,94.0,95.0,96.0,97.0,98.0,99.0,Type_1,Type_2,Type_3
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-05,1,0,1.0,0.035961,0.434149,0.050100,0.840500,0.405118,3,0.630267,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,26.0,0.016934,0.434149,0.050100,0.840500,0.405118,3,0.630267,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,17.0,0.019079,0.434149,0.050100,0.840500,0.405118,3,0.630267,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,45.0,0.000054,0.434149,0.050100,0.840500,0.405118,3,0.630267,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,28.0,0.001566,0.434149,0.050100,0.840500,0.405118,3,0.630267,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-10-26,45,0,13.0,0.037859,0.595988,0.706413,0.654796,0.458884,2,0.451136,...,0,0,0,0,0,0,0,0,1,0
2012-10-26,45,0,16.0,0.003838,0.595988,0.706413,0.654796,0.458884,2,0.451136,...,0,0,0,0,0,0,0,0,1,0
2012-10-26,45,0,32.0,0.005961,0.595988,0.706413,0.654796,0.458884,2,0.451136,...,0,0,0,0,0,0,0,0,1,0


In [97]:
df_encoded = df_encoded.drop(["Dept", "Type"], axis = 1)
df_encoded

,Store,IsHoliday,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,Size,Total_Markdown,1.0,...,93.0,94.0,95.0,96.0,97.0,98.0,99.0,Type_1,Type_2,Type_3
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-05,1,0,0.035961,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,1,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,0.016934,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,0.019079,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,0.000054,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,0.001566,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-10-26,45,0,0.037859,0.595988,0.706413,0.654796,0.458884,0.451136,0.032691,0,...,0,0,0,0,0,0,0,0,1,0
2012-10-26,45,0,0.003838,0.595988,0.706413,0.654796,0.458884,0.451136,0.032691,0,...,0,0,0,0,0,0,0,0,1,0
2012-10-26,45,0,0.005961,0.595988,0.706413,0.654796,0.458884,0.451136,0.032691,0,...,0,0,0,0,0,0,0,0,1,0


In [98]:
df_encoded

,Store,IsHoliday,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,Size,Total_Markdown,1.0,...,93.0,94.0,95.0,96.0,97.0,98.0,99.0,Type_1,Type_2,Type_3
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-05,1,0,0.035961,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,1,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,0.016934,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,0.019079,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,0.000054,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,0.001566,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-10-26,45,0,0.037859,0.595988,0.706413,0.654796,0.458884,0.451136,0.032691,0,...,0,0,0,0,0,0,0,0,1,0
2012-10-26,45,0,0.003838,0.595988,0.706413,0.654796,0.458884,0.451136,0.032691,0,...,0,0,0,0,0,0,0,0,1,0
2012-10-26,45,0,0.005961,0.595988,0.706413,0.654796,0.458884,0.451136,0.032691,0,...,0,0,0,0,0,0,0,0,1,0


In [99]:
df_encoded.to_csv("walmart_2.csv")

In [ ]:
# pd.merge(df, dept_encoded, left_index=True, right_index=True)

In [51]:
# df['Weekly_Sales'].fillna(df['purch_amt'].median(), inplace=True)

In [52]:
# store1 = df[df["Store"] == 1]
# store1

In [53]:
# store1 = store1.sort_index()
# store1.interpolate(method ='linear', limit_direction ='forward')

In [100]:
stores_data = []
for store in np.unique(df_encoded["Store"]):
    store_data = df_encoded[df_encoded["Store"] == store]
    store_data = store_data.sort_index()
    store_data = store_data.interpolate(method ='linear', limit_direction ='forward')
    stores_data.append(store_data)

In [101]:
stores_data[1]

,Store,IsHoliday,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,Size,Total_Markdown,1.0,...,93.0,94.0,95.0,96.0,97.0,98.0,99.0,Type_1,Type_2,Type_3
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-05,2,0,0.050547,0.413405,0.050100,0.837102,0.426011,0.906277,0.0000,1,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,2,0,0.019355,0.413405,0.050100,0.837102,0.426011,0.906277,0.0000,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,2,0,0.153258,0.413405,0.050100,0.837102,0.426011,0.906277,0.0000,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,2,0,0.041053,0.413405,0.050100,0.837102,0.426011,0.906277,0.0000,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,2,0,0.002298,0.413405,0.050100,0.837102,0.426011,0.906277,0.0000,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-10-26,2,0,0.050195,0.703033,0.518036,0.958935,0.219571,0.906277,0.0603,0,...,0,0,0,0,0,0,0,0,0,1
2012-10-26,2,0,0.023153,0.703033,0.518036,0.958935,0.219571,0.906277,0.0603,0,...,0,0,0,0,0,0,0,0,0,1
2012-10-26,2,0,0.018130,0.703033,0.518036,0.958935,0.219571,0.906277,0.0603,0,...,0,0,0,0,0,0,0,0,0,1


In [102]:
stores_data[33].isna().sum()

Store           0
IsHoliday       0
Weekly_Sales    0
Temperature     0
Fuel_Price      0
               ..
98.0            0
99.0            0
Type_1          0
Type_2          0
Type_3          0
Length: 93, dtype: int64

In [103]:
# store1 = stores_data[1].copy()
# to_nomalize = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Total_Markdown']
# store1[to_nomalize] = MinMaxScaler().fit_transform(store1[to_nomalize])
# store1

In [22]:
# to_nomalize = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Total_Markdown']
# for store_data in stores_data:
#     store_data[to_nomalize] = MinMaxScaler().fit_transform(store_data[to_nomalize])

In [104]:
stores_data[26]

,Store,IsHoliday,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,Size,Total_Markdown,1.0,...,93.0,94.0,95.0,96.0,97.0,98.0,99.0,Type_1,Type_2,Type_3
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-05,27,0,0.046622,0.286204,0.241483,0.091812,0.417673,0.916437,0.00000,1,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,27,0,0.048897,0.286204,0.241483,0.091812,0.417673,0.916437,0.00000,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,27,0,0.070240,0.286204,0.241483,0.091812,0.417673,0.916437,0.00000,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,27,0,0.013247,0.286204,0.241483,0.091812,0.417673,0.916437,0.00000,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,27,0,0.026458,0.286204,0.241483,0.091812,0.417673,0.916437,0.00000,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-10-26,27,0,0.002864,0.597358,0.801102,0.165055,0.394959,0.916437,0.06834,0,...,0,0,0,0,0,0,0,0,0,1
2012-10-26,27,0,0.091036,0.597358,0.801102,0.165055,0.394959,0.916437,0.06834,0,...,0,1,0,0,0,0,0,0,0,1
2012-10-26,27,0,0.000862,0.597358,0.801102,0.165055,0.394959,0.916437,0.06834,0,...,0,0,0,0,0,0,0,0,0,1


# Writing CSV

In [105]:
import os
path = "store-data"
print("----------------- Starting Execution ---------------")
for i in range(1, 46):
    dt = stores_data[i-1]
    try:
        os.makedirs(path)
    except:
        pass
    dt.to_csv(f"{path}/Store_{i}.csv")
    print(f"Created: Store_{i}.csv")
print("-------------------- End of Excution ----------------")

----------------- Starting Execution ---------------
Created: Store_1.csv
Created: Store_2.csv
Created: Store_3.csv
Created: Store_4.csv
Created: Store_5.csv
Created: Store_6.csv
Created: Store_7.csv
Created: Store_8.csv
Created: Store_9.csv
Created: Store_10.csv
Created: Store_11.csv
Created: Store_12.csv
Created: Store_13.csv
Created: Store_14.csv
Created: Store_15.csv
Created: Store_16.csv
Created: Store_17.csv
Created: Store_18.csv
Created: Store_19.csv
Created: Store_20.csv
Created: Store_21.csv
Created: Store_22.csv
Created: Store_23.csv
Created: Store_24.csv
Created: Store_25.csv
Created: Store_26.csv
Created: Store_27.csv
Created: Store_28.csv
Created: Store_29.csv
Created: Store_30.csv
Created: Store_31.csv
Created: Store_32.csv
Created: Store_33.csv
Created: Store_34.csv
Created: Store_35.csv
Created: Store_36.csv
Created: Store_37.csv
Created: Store_38.csv
Created: Store_39.csv
Created: Store_40.csv
Created: Store_41.csv
Created: Store_42.csv
Created: Store_43.csv
Created: S

In [ ]:
# sm.tsa.seasonal_decompose()

# Reading CSV

In [106]:
stores_data = []
path = "store-data"
print("--------------- Starting Execution -------------")
for i in range(1, 46):
    store = pd.read_csv(f"{path}/Store_{i}.csv")
    store["Date"] = pd.to_datetime(store["Date"])
    store = store.set_index("Date")
    print(f"Read: Store_{i}.csv")
    stores_data.append(store)
print("---------------- End of Execution -------------") 

--------------- Starting Execution -------------
Read: Store_1.csv
Read: Store_2.csv
Read: Store_3.csv
Read: Store_4.csv
Read: Store_5.csv
Read: Store_6.csv
Read: Store_7.csv
Read: Store_8.csv
Read: Store_9.csv
Read: Store_10.csv
Read: Store_11.csv
Read: Store_12.csv
Read: Store_13.csv
Read: Store_14.csv
Read: Store_15.csv
Read: Store_16.csv
Read: Store_17.csv
Read: Store_18.csv
Read: Store_19.csv
Read: Store_20.csv
Read: Store_21.csv
Read: Store_22.csv
Read: Store_23.csv
Read: Store_24.csv
Read: Store_25.csv
Read: Store_26.csv
Read: Store_27.csv
Read: Store_28.csv
Read: Store_29.csv
Read: Store_30.csv
Read: Store_31.csv
Read: Store_32.csv
Read: Store_33.csv
Read: Store_34.csv
Read: Store_35.csv
Read: Store_36.csv
Read: Store_37.csv
Read: Store_38.csv
Read: Store_39.csv
Read: Store_40.csv
Read: Store_41.csv
Read: Store_42.csv
Read: Store_43.csv
Read: Store_44.csv
Read: Store_45.csv
---------------- End of Execution -------------


In [107]:
len(stores_data)

45

In [108]:
stores_data[0]

,Store,IsHoliday,Weekly_Sales,Temperature,Fuel_Price,CPI,Unemployment,Size,Total_Markdown,1.0,...,93.0,94.0,95.0,96.0,97.0,98.0,99.0,Type_1,Type_2,Type_3
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-05,1,0,0.035961,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,1,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,0.016934,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,0.019079,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,0.000054,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,0,...,0,0,0,0,0,0,0,0,0,1
2010-02-05,1,0,0.001566,0.434149,0.050100,0.840500,0.405118,0.630267,0.000000,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-10-26,1,0,0.055566,0.696869,0.518036,0.962552,0.258194,0.630267,0.031062,0,...,0,0,0,0,0,0,0,0,0,1
2012-10-26,1,0,0.096123,0.696869,0.518036,0.962552,0.258194,0.630267,0.031062,0,...,1,0,0,0,0,0,0,0,0,1
2012-10-26,1,0,0.004820,0.696869,0.518036,0.962552,0.258194,0.630267,0.031062,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
X_pred = model.predict(X_train)
X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
X_pred = pd.DataFrame(X_pred, columns=train.columns)
X_pred.index = train.index

scored = pd.DataFrame(index=train.index)
X_train1 = X_train.reshape(X_train.shape[0], X_train.shape[2])
scored['Loss_mae'] = np.mean(np.abs(X_pred-X_train1), axis = 1)
scored['Threshold'] = 0.11
scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']

anomalies = scored[scored.Anomaly == True]
anomalies.head(20)